In [5]:
import torch
import torch.nn as nn

In [6]:
# gpt-2 configurations
GPT_CONFIG_124= {
    'vocab_size' :50257,
    'context_length': 1024,
    'emb_dim':768,
    'n_heads':12,
    'n_layers':12,
    'drop_rate':0.1,
    'qkv_bias':False
}

In [7]:
#GELU Activation function
class GELU(nn.Module):
    def __init__(self,):
        super().__init__()
    def forward(self, x):
        return 0.5*x*(1+torch.tanh(torch.sqrt(torch.tensor(2.0/torch.pi))*(x+0.0447*torch.pow(x,3))))

In [8]:
#Feed Forward Neural network
class FeedForward(nn.Module):
    def __init__(self, cfg):
        super().__init__()
        self.layers = nn.Sequential(nn.Linear(cfg['emb_dim'], 4*cfg['emb_dim']), #incresing dimesion
                                    GELU(),
                                    nn.Linear(4*cfg['emb_dim'],cfg['emb_dim']))  #coming back to original dimension
    def forward(self,x):
        return self.layers(x)

In [9]:
# Layer Normalization
class LayerNorm(nn.Module):
    def __init__(self, emd_dim):
        super().__init__()
        self.eps = 1e-5
        self.scale = nn.Parameter(torch.ones(emd_dim))
        self.shift = nn.Parameter(torch.zeros(emd_dim))
    def forward(self, x):
        mean = x.mean(dim = -1, keepdim =True)
        var = x.var(dim =-1, keepdim = True, unbiased= False)
        norm_x = (x-mean)/(torch.sqrt(var+self.eps))
        return self.scale*norm_x + self.shift

In [10]:
class MultiHeadAttention(nn.Module):
    def __init__(self,d_in,d_out,context_length,dropout,num_heads,qkv_bias = False):
        super().__init__()
        assert(d_out % num_heads==0),\
            "d_out must be divisible by nums head"
        
        self.d_out = d_out
        self.num_head = num_heads
        self.head_dim = d_out//num_heads
        # self.d_in = d_in
        self.w_query = nn.Linear(d_in,d_out, bias=qkv_bias)
        self.w_key = nn.Linear(d_in,d_out, bias=qkv_bias)
        self.w_value = nn.Linear(d_in,d_out, bias=qkv_bias)

        self.out_proj = nn.Linear(d_out, d_out)  # Linear layer to combine head outputs
        self.dropout = nn.Dropout(dropout)
        self.register_buffer('mask',torch.triu(torch.ones(context_length,context_length),diagonal=1))

    def forward(self,x):
        b,num_token,d_in = x.shape

        keys = self.w_key(x) # Shape: (b, num_tokens, d_out)
        queries = self.w_query(x)
        values = self.w_value(x)

        keys = keys.view(b,num_token,self.num_head,self.head_dim)
        queries = queries.view(b,num_token,self.num_head,self.head_dim)
        values = values.view(b,num_token,self.num_head,self.head_dim)

        #grouping by num_heads
        keys = keys.transpose(1,2)
        values = values.transpose(1,2)
        queries = queries.transpose(1,2)

        # calculating attention score
        attn_score = queries @ keys.transpose(2,3)

        # calculating attention weigths,masking, scaling and dropout
        mask_bool = self.mask.bool()[:num_token,:num_token]
        attn_score= attn_score.masked_fill_(mask_bool, - torch.inf)
        attn_weight = torch.softmax(attn_score/keys.shape[-1]**0.5, dim=-1)
        attn_weight = self.dropout(attn_weight)
        
        #calculating the context vector
        context_vector = attn_weight @ values #ntokn x ntoken * ntoken x head_dim
        # trasposing to get all the context vextor togeth
        context_vector = context_vector.transpose(1,2)

        # combining heads 
        context_vector = context_vector.contiguous().view(b,num_token,self.d_out)
        context_vector = self.out_proj(context_vector) # optional projection
        return context_vector

In [11]:
class TransformerBlock(nn.Module):
    def __init__(self,cfg):
        super().__init__()
        self.attn = MultiHeadAttention(d_in=cfg['emb_dim'], d_out= cfg['emb_dim'],
                                       context_length=cfg['context_length'],
                                       num_heads=cfg['n_heads'], dropout= cfg['drop_rate'],
                                       qkv_bias= cfg['qkv_bias'])
        self.ff = FeedForward(cfg=cfg)
        self.norm1 = LayerNorm(emd_dim=cfg['emb_dim'])
        self.norm2 = LayerNorm(emd_dim=cfg['emb_dim'])

        self.drop_shortcut  = nn.Dropout(cfg['drop_rate'])
    
    def forward(self,x):
        shortcut = x
        x= self.norm1(x)
        x = self.attn(x)
        x = self.drop_shortcut(x)

        x = x+ shortcut

        shortcut =x
        x = self.norm2(x)
        x = self.ff(x)
        x = self.drop_shortcut(x)
        x = x + shortcut

        return x


In [12]:
class GPTModel(nn.Module):
    def __init__(self,cfg):
        super().__init__()
        self.tok_emb = nn.Embedding(cfg['vocab_size'], cfg['emb_dim'])
        self.pos_emb = nn.Embedding(cfg['context_length'], cfg['emb_dim'])
        self.drop_emb = nn.Dropout(cfg['drop_rate'])
        self.trf_blocks = nn.Sequential(*[TransformerBlock(cfg) for _ in range(cfg['n_layers'])])
        self.final_norm  = LayerNorm(cfg['emb_dim'])
        self.out_head = nn.Linear(cfg['emb_dim'],cfg['vocab_size'], bias=False)
    def forward(self,in_idx):
        batch_size, seq_len = in_idx.shape
        token_embded = self.tok_emb(in_idx)
        pos_embded = self.pos_emb(torch.arange(seq_len, device=in_idx.device))
        x = token_embded+pos_embded
        x = self.drop_emb(x)
        x = self.trf_blocks(x)
        x = self.final_norm(x)
        logits = self.out_head(x)
        return logits



In [13]:
# gpt-2 configurations
GPT_CONFIG_124M= {
    'vocab_size' :50257,
    'context_length': 256,
    'emb_dim':768,
    'n_heads':12,
    'n_layers':12,
    'drop_rate':0.1,
    'qkv_bias':False
}

In [14]:
torch.manual_seed(123)
model = GPTModel(GPT_CONFIG_124M)

In [15]:
def generate_text_simple(model, idx, max_words, context_size):

    for _  in range(max_words):
        idx_cond = idx[:,-context_size:]
        with torch.no_grad():
            logits = model(idx_cond)
        logits = logits[:,-1,:]
        probas = torch.softmax(logits,dim=-1)
        idx_next = torch.argmax(probas,dim=-1,keepdim=True)
        idx = torch.cat((idx,idx_next), dim=1)
    return idx

In [16]:
import tiktoken
tokenizer = tiktoken.get_encoding("gpt2")

In [17]:
def text_to_token(text, tokenizer):
    encoded = tokenizer.encode(text, allowed_special={'<|endoftext|>'})
    encoded_tensor = torch.tensor(encoded).unsqueeze(0)
    return encoded_tensor
def token_to_text(ids, tokenizer):
    flat = ids.squeeze(0)
    return tokenizer.decode(flat.tolist())

In [18]:
batch = []
out_batch =[]
txt1 = "Every effort moves"
txt2 = "I really like"
batch.append(torch.tensor(tokenizer.encode(txt1)))
batch.append(torch.tensor(tokenizer.encode(txt2)))
batch = torch.stack(batch, dim=0)
print(batch)


tensor([[6109, 3626, 6100],
        [  40, 1107,  588]])


In [19]:
output = torch.tensor([[3626,6100,345],
                       [1107,588,11311]])
output

tensor([[ 3626,  6100,   345],
        [ 1107,   588, 11311]])

In [20]:
with torch.no_grad():
    logits = model(batch)
probas = torch.softmax(logits,dim=-1)
probas.shape

torch.Size([2, 3, 50257])

In [21]:
token_ids = torch.argmax(probas,dim=-1,keepdim=True)
print(token_ids)

tensor([[[31093],
         [16031],
         [44376]],

        [[49906],
         [ 3684],
         [38185]]])


In [22]:
print("Targrt batch1:", token_to_text(output[0],tokenizer))
print("prediction batch1:", token_to_text(token_ids[0].flatten(),tokenizer))

Targrt batch1:  effort moves you
prediction batch1:  moons saves inaction


### Cross Entropy Loss

In [23]:
text_idx = 0
target_proba_1 = probas[text_idx, [0,1,2], output[text_idx]]
print("Text 1:", target_proba_1)

text_idx = 1
target_proba_2 = probas[text_idx, [0,1,2], output[text_idx]]
print("Text 2: ", target_proba_2)

Text 1: tensor([3.1416e-05, 2.0867e-05, 1.1188e-05])
Text 2:  tensor([1.3012e-05, 6.0395e-05, 4.8063e-06])


In [24]:
log_proba = torch.log(torch.cat((target_proba_1,target_proba_2)))
print(log_proba)
avg_log_proba = torch.mean(log_proba)
print(avg_log_proba)
neg_log_proba = avg_log_proba*-1
print(neg_log_proba)

tensor([-10.3682, -10.7774, -11.4007, -11.2496,  -9.7146, -12.2456])
tensor(-10.9593)
tensor(10.9593)


In [25]:
# doing these all with onliner 
target_flat = output.flatten()
logits_flat = logits.flatten(0,1)
loss = torch.nn.functional.cross_entropy(logits_flat,target_flat)
print(loss)

tensor(10.9593)


### Perplexity

In [26]:
p_loss = torch.exp(loss)

In [27]:
p_loss

tensor(57488.4375)

### Calculating the loss for the entire dataset

In [28]:
import os
with open("the-verdict.txt", 'r', encoding= 'utf-8') as f:
    raw_data = f.read()
# print(raw_data)

In [29]:
raw_data[:25]

'I HAD always thought Jack'

In [30]:
print(len(raw_data))

20479


In [32]:
print("No of tokens:" , len(tokenizer.encode(raw_data)))

No of tokens: 5145


In [37]:
from torch.utils.data import Dataset, DataLoader

class GPTDatasetV1(Dataset):
    def __init__(self,text, tokenizer, max_length, stride):
        self.input_ids = []
        self.target_ids= []

        token_ids = tokenizer.encode(text, allowed_special={"<|endoftext|>"})
        for i in range(0, len(token_ids)-max_length,stride):
            input_chunk = token_ids[i:i+max_length]
            target_chunck = token_ids[i+1:i+1+max_length]
            self.input_ids.append(torch.tensor(input_chunk))
            self.target_ids.append(torch.tensor(target_chunck))
    def __len__(self):
        return len(self.input_ids)
    def __getitem__(self, index):
        return self.input_ids[index], self.target_ids[index]

In [38]:
def create_dataloader(text, batch_size=4, max_length = 256, stride = 256, shuffle = True, drop_last = True, num_workers = 0):
    tokenizer = tiktoken.get_encoding("gpt2")
    dataset = GPTDatasetV1(text=text, tokenizer=tokenizer,max_length=max_length,stride=stride)
    #Creating dataloader
    dataloader = DataLoader(dataset=dataset,
                            batch_size=batch_size,
                            shuffle=shuffle,
                            drop_last=drop_last,
                            num_workers=num_workers)
    return dataloader

In [39]:
# gpt-2 configurations
GPT_CONFIG_124M= {
    'vocab_size' :50257,
    'context_length': 256,
    'emb_dim':768,
    'n_heads':12,
    'n_layers':12,
    'drop_rate':0.1,
    'qkv_bias':False
}

In [40]:
train_ratio = 0.90
split_idx = int(train_ratio*len(raw_data))
train_data = raw_data[:split_idx]
test_data = raw_data[split_idx:]

torch.manual_seed(123)
train_loader = create_dataloader(
    text=train_data,
    batch_size=2,
    max_length= GPT_CONFIG_124M['context_length'],
    stride= GPT_CONFIG_124M['context_length'],
    drop_last= True,
    shuffle= True,
    num_workers=0
)

val_loader = create_dataloader(
    text=test_data,
    batch_size=2,
    max_length= GPT_CONFIG_124M['context_length'],
    stride= GPT_CONFIG_124M['context_length'],
    drop_last= True,
    shuffle= True,
    num_workers=0
)

In [42]:
for x,y in train_loader:
    print(x.shape, y.shape)
print("Output shape;")
for x,y in  val_loader:
    print(x.shape, y.shape)

torch.Size([2, 256]) torch.Size([2, 256])
torch.Size([2, 256]) torch.Size([2, 256])
torch.Size([2, 256]) torch.Size([2, 256])
torch.Size([2, 256]) torch.Size([2, 256])
torch.Size([2, 256]) torch.Size([2, 256])
torch.Size([2, 256]) torch.Size([2, 256])
torch.Size([2, 256]) torch.Size([2, 256])
torch.Size([2, 256]) torch.Size([2, 256])
torch.Size([2, 256]) torch.Size([2, 256])
Output shape;
torch.Size([2, 256]) torch.Size([2, 256])


In [43]:
#counting the train and validation tokens
train_token = 0
val_token = 0
for input_batch, out_batch in train_loader:
    train_token += input_batch.numel()
for x, y in val_loader:
    val_token += x.numel()
print("Input tokens count:" , train_token)
print("NO of validation token :", val_token)
print("total tokens :", train_token+val_token)

Input tokens count: 4608
NO of validation token : 512
total tokens : 5120


In [45]:
class GPTModel(nn.Module):
    def __init__(self,cfg):
        super().__init__()
        self.tok_emb = nn.Embedding(cfg['vocab_size'], cfg['emb_dim'])
        self.pos_emb = nn.Embedding(cfg['context_length'], cfg['emb_dim'])
        self.drop_emb = nn.Dropout(cfg['drop_rate'])
        self.trf_blocks = nn.Sequential(*[TransformerBlock(cfg) for _ in range(cfg['n_layers'])])
        self.final_norm  = LayerNorm(cfg['emb_dim'])
        self.out_head = nn.Linear(cfg['emb_dim'],cfg['vocab_size'], bias=False)
    def forward(self,in_idx):
        batch_size, seq_len = in_idx.shape
        token_embded = self.tok_emb(in_idx)
        pos_embded = self.pos_emb(torch.arange(seq_len, device=in_idx.device))
        
        x = token_embded+pos_embded
        x = self.drop_emb(x)
        x = self.trf_blocks(x)
        x = self.final_norm(x)
        logits = self.out_head(x)
        return logits



In [47]:
torch.manual_seed(123)
model = GPTModel(GPT_CONFIG_124M)


In [48]:
def calc_loss_batch(input_batch,target_batch,model, device):
    input_batch , target_batch = input_batch.to(device), target_batch.to(device)
    logits = model(input_batch)
    loss = torch.nn.functional.cross_entropy(logits.flatten(0,1), target_batch.flatten())
    return loss

def calc_loss_score( data_loader, model, device, num_batches = None):
    total_loss = 0
    if len(data_loader) ==0:
        return float("nan")
    elif num_batches is None:
        num_batches = len(data_loader)
    else:
        num_batches = min(len(data_loader), num_batches)

    for i ,(input_batch, target_batch) in enumerate(data_loader):
        if i<num_batches:
            loss = calc_loss_batch(input_batch=input_batch, target_batch=target_batch, model=model, device=device)
            total_loss += loss.item()
        else:
            break

    return total_loss/num_batches
            
    

In [49]:
#calculating the loss for both the train_set and the validation_set
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
with torch.no_grad():
    train_loss = calc_loss_score(train_loader, model=model, device=device)
    val_loss = calc_loss_score(val_loader, model=model, device=device)

In [51]:
print("train_loss: ", train_loss)
print("Validation loss", val_loss)

train_loss:  10.995637469821506
Validation loss 10.996355056762695


In [52]:
#calculating the loss for harry potter book

with open('01HarryPotterandtheSorcerers.txt', 'r', encoding='utf-8') as f:
    harry_potter  = f.read()

In [53]:
train_ratio = 0.90
split_idx = int(train_ratio*len(harry_potter))
train_data = harry_potter[:split_idx]
test_data = harry_potter[split_idx:]

torch.manual_seed(123)
train_loader = create_dataloader(
    text=train_data,
    batch_size=2,
    max_length= GPT_CONFIG_124M['context_length'],
    stride= GPT_CONFIG_124M['context_length'],
    drop_last= True,
    shuffle= True,
    num_workers=0
)

val_loader = create_dataloader(
    text=test_data,
    batch_size=2,
    max_length= GPT_CONFIG_124M['context_length'],
    stride= GPT_CONFIG_124M['context_length'],
    drop_last= True,
    shuffle= True,
    num_workers=0
)

In [54]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
with torch.no_grad():
    train_loss = calc_loss_score(train_loader, model=model, device=device)
    val_loss = calc_loss_score(val_loader, model=model, device=device)

In [55]:
print("train_loss: ", train_loss)
print("Validation loss", val_loss)

train_loss:  10.966798134900014
Validation loss 10.9551074107488
